In [1]:
!pip install opencv-python opencv-contrib-python numpy matplotlib pillow tqdm

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [ ]:
# # rename files in folder
# import os
# import re
# import shutil
#
# folder = './files'
# for filename in os.listdir(folder):
#     test_re = re.compile(r'\((\d*)\).png')
#     match = test_re.search(filename)
#     print(match)
#     if match:
#         new_name = match.group(1) + '.png'
#         print(filename, '->', new_name)
#         os.rename(os.path.join(folder, filename), os.path.join(folder, new_name))

In [ ]:
#%matplotlib inline

from numpy import ndarray, dtype, generic
from matplotlib import pyplot as plt

import numpy as np
import os
import cv2

folder_base = './files'

IMG_END = 226

output_folder = './output'
count = 0

slice_length = 20


def get_output_filename():
    global count
    count += 1
    return os.path.join(output_folder, '{}.png'.format(count))


def check_part(part_img: ndarray[int, dtype[generic]]):
    hsv = cv2.cvtColor(part_img, cv2.COLOR_BGR2HSV)
    lower_red = np.array([0, 0, 0])
    upper_red = np.array([80, 255, 40])
    mask = cv2.inRange(hsv, lower_red, upper_red)

    res = cv2.bitwise_and(hsv, hsv, mask=mask)
    value = cv2.split(res)[2]
    r_num = 0
    for i in range(value.shape[0]):
        for j in range(value.shape[1]):
            if value[i, j] > 0:
                r_num += 1

    if r_num > 10:
        return True
    else:
        return False


for i in range(1, IMG_END + 1):
    target_file = os.path.join(folder_base, str(i) + '.png')
    img = cv2.imread(target_file)
    # get image size
    height, width, channel = img.shape

    part_start = -1
    part_end = -1

    for scan_start in range(200, height - 200, slice_length):
        scan_end = scan_start + slice_length
        if scan_end > height - 200:
            scan_end = height - 200
        part = img[scan_start:scan_end, :]
        should = check_part(part)
        if should:
            if part_start == -1:
                part_start = scan_start
            else:
                part_end = scan_start
        else:
            if part_start != -1:
                part_end = scan_start
                save_img = img[part_start - 100:part_end + 100, :]
                num = get_output_filename()
                print("Capture: {}".format(num))
                cv2.imwrite(num, save_img)
                part_start = -1
                part_end = -1

    if part_start != -1:
        part_end = height - 200
        save_img = img[part_start - 100:part_end + 100, :]
        num = get_output_filename()
        print("Capture: {}".format(num))
        cv2.imwrite(num, save_img)

        # plt.imshow(part)
        # plt.show()

    # cv2.namedWindow("img", cv2.WINDOW_NORMAL)
    # cv2.imshow('img', None)
    # cv2.waitKey()
    # cv2.destroyAllWindows()
    origin = img.copy()

Capture: ./output\1.png
Capture: ./output\2.png
Capture: ./output\3.png
Capture: ./output\4.png
Capture: ./output\5.png
Capture: ./output\6.png
Capture: ./output\7.png
Capture: ./output\8.png
Capture: ./output\9.png
Capture: ./output\10.png
Capture: ./output\11.png
Capture: ./output\12.png
Capture: ./output\13.png
Capture: ./output\14.png
Capture: ./output\15.png
Capture: ./output\16.png
Capture: ./output\17.png
Capture: ./output\18.png
Capture: ./output\19.png
Capture: ./output\20.png
Capture: ./output\21.png
Capture: ./output\22.png
Capture: ./output\23.png
Capture: ./output\24.png
Capture: ./output\25.png
Capture: ./output\26.png
Capture: ./output\27.png
Capture: ./output\28.png
Capture: ./output\29.png
Capture: ./output\30.png
Capture: ./output\31.png
Capture: ./output\32.png
Capture: ./output\33.png
Capture: ./output\34.png
Capture: ./output\35.png
Capture: ./output\36.png
Capture: ./output\37.png
Capture: ./output\38.png
Capture: ./output\39.png
Capture: ./output\40.png
Capture: 